In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from lib import Datasets
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [2]:
model_name = "meta-llama/Llama-2-7b-hf"
# model_name = "distilgpt2"
# model_name = "roberta-base"
device = 'cuda:0'

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
# model_name = "roberta-base"

# model_checkpoint = "NousResearch/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

for name, p in model.named_parameters():
    print(name, p.shape)

2025-02-11 22:20:32.531108: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-11 22:20:32.546508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739301632.563378 3430134 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739301632.568657 3430134 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 22:20:32.589715: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model.embed_tokens.weight torch.Size([32000, 4096])
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.k_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.v_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.0.mlp.gate_proj.weight torch.Size([11008, 4096])
model.layers.0.mlp.up_proj.weight torch.Size([11008, 4096])
model.layers.0.mlp.down_proj.weight torch.Size([4096, 11008])
model.layers.0.input_layernorm.weight torch.Size([4096])
model.layers.0.post_attention_layernorm.weight torch.Size([4096])
model.layers.1.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.k_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.v_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.1.mlp.gate_proj.weight torch.Size([11008, 4096])
model.layers.1.mlp.up_proj.weight torch.Size([11008, 4096])
model.l

In [3]:
b = 8
# lm_datasets = Datasets.LoadWikiOther(tokenizer, b)

In [4]:
# print(lm_datasets)

In [5]:
# import torch
# from lib import Train

# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Train.train_model(model, lm_datasets, optimizer, tokenizer)


In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    f"./workout/keke-test",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=10,
    logging_dir=f"./logs/kekek-test",
    push_to_hub=False,
    save_strategy="epoch",
    # use_cpu=True,
    # bf16=True,
)

In [7]:
lm_datasets = Datasets.LoadWiki(tokenizer, 10, 128)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [8]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'. Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n<s>  The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjust'

In [9]:
# trainer.train()

In [10]:


# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [11]:
import math
import time

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.4f}")

# while True:
#     try:
#         eval_results = trainer.evaluate()
#         print(f"Perplexity: {math.exp(eval_results['eval_loss']):.4f}")
#         break
#     except Exception as e:
#         print("Вы обосрались")
#         print(e)
#         time.sleep(1)

Perplexity: 15.1078
